# Steps Done Already

### Downloading the Dataset and Uploading it to MongoDB

First, we downloaded the tsv file from UniProt website and created a python program to convert it to csv file. Since the size of the file was large, mongodb was not able to upload it to the database, therefore we split the database into 8 chunks of 10000 entries, and then fed them separately to mongodb



In [2]:
import numpy as np
from itertools import combinations
import csv

In [3]:
import pandas as pd

#df = pd.read_csv("data/uniprotkb_AND_model_organism_10090_2024_11_21.csv")
df = pd.read_csv("data/moodle_file.tsv", delimiter="\t")


In [4]:
df.head()

,Entry,Entry Name,Protein names,Gene Names,Organism,Sequence,EC number,InterPro
0,A0A075F5C6,A0A075F5C6_MOUSE,Heat shock factor 1 (Heat shock transcription ...,Hsf1,Mus musculus (Mouse),MDLAVGPGAAGPSNVPAFLTKLWTLVSDPDTDALICWSPSGNSFHV...,NaN,IPR000232;IPR027725;IPR010542;IPR036388;IPR036...
1,A0A087WPF7,AUTS2_MOUSE,Autism susceptibility gene 2 protein homolog,Auts2 Kiaa0442,Mus musculus (Mouse),MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAP...,NaN,IPR023246;
2,A0A087WPU4,A0A087WPU4_MOUSE,FAT atypical cadherin 1,Fat1,Mus musculus (Mouse),XRPYFDSKLNKNIYSDIPPQVPVRPISYTPSIPSDSRNNLDRNSFE...,NaN,NaN
3,A0A087WRK1,A0A087WRK1_MOUSE,"Predicted gene, 20814 (Predicted gene, 20855) ...",Gm20905 Gm20814 Gm20835 Gm20850 Gm20855 Gm2086...,Mus musculus (Mouse),MRRMALKKLKVIPKEGYLLLLDFDDEDDDIKVSEEALSEVKSPAFD...,NaN,IPR051443;IPR006888;
4,A0A087WRT4,A0A087WRT4_MOUSE,FAT atypical cadherin 1,Fat1,Mus musculus (Mouse),MGRHLTLLLLLLLFLQQFGDSDGSQRLEPTPPIQFTHFQYNVTVHE...,NaN,IPR039808;IPR002126;IPR015919;IPR020894;IPR013...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85882 entries, 0 to 85881
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Entry          85882 non-null  object
 1   Entry Name     85882 non-null  object
 2   Protein names  85882 non-null  object
 3   Gene Names     82540 non-null  object
 4   Organism       85882 non-null  object
 5   Sequence       85882 non-null  object
 6   EC number      10458 non-null  object
 7   InterPro       73862 non-null  object
dtypes: object(8)
memory usage: 5.2+ MB


In [6]:
df.describe()

,Entry,Entry Name,Protein names,Gene Names,Organism,Sequence,EC number,InterPro
count,85882,85882,85882,82540,85882,85882,10458,73862
unique,85882,85882,44475,36800,1,82968,1773,21481
top,A0A075F5C6,A0A075F5C6_MOUSE,Uncharacterized protein,H2-K1,Mus musculus (Mouse),WFLEYSTSECHFYNGTQRVRLLERYFYNLEENLRFDSDVGEFRAVT...,2.7.11.1,IPR000276;IPR017452;IPR000725;
freq,1,1,4118,289,85882,12,675,1160


In [5]:
df = df.drop_duplicates()
df.describe()

,Entry,Entry Name,Protein names,Gene Names,Organism,Sequence,EC number,InterPro
count,85882,85882,85882,82540,85882,85882,10458,73862
unique,85882,85882,44475,36800,1,82968,1773,21481
top,A0A075F5C6,A0A075F5C6_MOUSE,Uncharacterized protein,H2-K1,Mus musculus (Mouse),WFLEYSTSECHFYNGTQRVRLLERYFYNLEENLRFDSDVGEFRAVT...,2.7.11.1,IPR000276;IPR017452;IPR000725;
freq,1,1,4118,289,85882,12,675,1160


In [25]:
print(df.isnull().sum())

Entry                0
Entry Name           0
Protein names        0
Gene Names        3342
Organism             0
Sequence             0
EC number        75424
InterPro         12020
dtype: int64


In [ ]:
#df = df.dropna()
#df = df.dropna(axis=1)
#df.describe()

In [9]:
print(df.isnull().sum())

Entry                0
Entry Name           0
Protein names        0
Gene Names        3342
Organism             0
Sequence             0
EC number        75424
InterPro         12020
dtype: int64


In [6]:
df['Gene Names'] = df['Gene Names'].fillna("Unknown")
df['EC number'] = df['EC number'].fillna("Not Available")
df = df.dropna(subset=['InterPro'])
print(df.isnull().sum())

Entry            0
Entry Name       0
Protein names    0
Gene Names       0
Organism         0
Sequence         0
EC number        0
InterPro         0
dtype: int64


In [10]:
df.describe()

,Entry,Entry Name,Protein names,Gene Names,Organism,Sequence,EC number,InterPro
count,73862,73862,73862,73862,73862,73862,73862,73862
unique,73862,73862,41087,33656,1,70969,1767,21481
top,A0A075F5C6,A0A075F5C6_MOUSE,Olfactory receptor,Unknown,Mus musculus (Mouse),WFLEYSTSECHFYNGTQRVRLLERYFYNLEENLRFDSDVGEFRAVT...,Not Available,IPR000276;IPR017452;IPR000725;
freq,1,1,1741,2444,73862,12,63447,1160


In [7]:
df['Domains'] = df['InterPro'].apply(lambda x: ','.join(x.split(';')) if isinstance(x, str) else '')
df['InterPro'] = df['InterPro'].apply(lambda x: x.split(';') if isinstance(x, str) else [])
df.describe()

,Entry,Entry Name,Protein names,Gene Names,Organism,Sequence,EC number,InterPro,Domains
count,73862,73862,73862,73862,73862,73862,73862,73862,73862
unique,73862,73862,41087,33656,1,70969,1767,21481,21481
top,A0A075F5C6,A0A075F5C6_MOUSE,Olfactory receptor,Unknown,Mus musculus (Mouse),WFLEYSTSECHFYNGTQRVRLLERYFYNLEENLRFDSDVGEFRAVT...,Not Available,"[IPR000276, IPR017452, IPR000725, ]","IPR000276,IPR017452,IPR000725,"
freq,1,1,1741,2444,73862,12,63447,1160,1160


In [22]:
df.head()

,Entry,Entry Name,Protein names,Gene Names,Organism,Sequence,EC number,InterPro,Domains
0,A0A075F5C6,A0A075F5C6_MOUSE,Heat shock factor 1 (Heat shock transcription ...,Hsf1,Mus musculus (Mouse),MDLAVGPGAAGPSNVPAFLTKLWTLVSDPDTDALICWSPSGNSFHV...,Not Available,"[IPR000232, IPR027725, IPR010542, IPR036388, I...","IPR000232,IPR027725,IPR010542,IPR036388,IPR036..."
1,A0A087WPF7,AUTS2_MOUSE,Autism susceptibility gene 2 protein homolog,Auts2 Kiaa0442,Mus musculus (Mouse),MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAP...,Not Available,"[IPR023246, ]","IPR023246,"
3,A0A087WRK1,A0A087WRK1_MOUSE,"Predicted gene, 20814 (Predicted gene, 20855) ...",Gm20905 Gm20814 Gm20835 Gm20850 Gm20855 Gm2086...,Mus musculus (Mouse),MRRMALKKLKVIPKEGYLLLLDFDDEDDDIKVSEEALSEVKSPAFD...,Not Available,"[IPR051443, IPR006888, ]","IPR051443,IPR006888,"
4,A0A087WRT4,A0A087WRT4_MOUSE,FAT atypical cadherin 1,Fat1,Mus musculus (Mouse),MGRHLTLLLLLLLFLQQFGDSDGSQRLEPTPPIQFTHFQYNVTVHE...,Not Available,"[IPR039808, IPR002126, IPR015919, IPR020894, I...","IPR039808,IPR002126,IPR015919,IPR020894,IPR013..."
5,A0A087WSP5,A0A087WSP5_MOUSE,Signal transducer and activator of transcription,Stat1,Mus musculus (Mouse),MSQWFELQQLDSKFLEQVHQLYDDSFPMEIRQYLAQWLEKQDWEHA...,Not Available,"[IPR008967, IPR000980, IPR036860, IPR001217, I...","IPR008967,IPR000980,IPR036860,IPR001217,IPR038..."


In [13]:
df

,Entry,Entry Name,Protein names,Gene Names,Organism,Sequence,EC number,InterPro
0,A0A075F5C6,A0A075F5C6_MOUSE,Heat shock factor 1 (Heat shock transcription ...,Hsf1,Mus musculus (Mouse),MDLAVGPGAAGPSNVPAFLTKLWTLVSDPDTDALICWSPSGNSFHV...,Not Available,"[IPR000232, IPR027725, IPR010542, IPR036388, I..."
1,A0A087WPF7,AUTS2_MOUSE,Autism susceptibility gene 2 protein homolog,Auts2 Kiaa0442,Mus musculus (Mouse),MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAP...,Not Available,"[IPR023246, ]"
3,A0A087WRK1,A0A087WRK1_MOUSE,"Predicted gene, 20814 (Predicted gene, 20855) ...",Gm20905 Gm20814 Gm20835 Gm20850 Gm20855 Gm2086...,Mus musculus (Mouse),MRRMALKKLKVIPKEGYLLLLDFDDEDDDIKVSEEALSEVKSPAFD...,Not Available,"[IPR051443, IPR006888, ]"
4,A0A087WRT4,A0A087WRT4_MOUSE,FAT atypical cadherin 1,Fat1,Mus musculus (Mouse),MGRHLTLLLLLLLFLQQFGDSDGSQRLEPTPPIQFTHFQYNVTVHE...,Not Available,"[IPR039808, IPR002126, IPR015919, IPR020894, I..."
5,A0A087WSP5,A0A087WSP5_MOUSE,Signal transducer and activator of transcription,Stat1,Mus musculus (Mouse),MSQWFELQQLDSKFLEQVHQLYDDSFPMEIRQYLAQWLEKQDWEHA...,Not Available,"[IPR008967, IPR000980, IPR036860, IPR001217, I..."
...,...,...,...,...,...,...,...,...
85874,Z4YN47,Z4YN47_MOUSE,SKI/DACH domain containing 1,Skida1,Mus musculus (Mouse),MGDLKSGFEEVDGVRLGYLIIKGKQMFALSQVFTDLLKNIPRTTVH...,Not Available,"[IPR009061, IPR052119, IPR003380, IPR037000, ]"
85875,Z4YN62,Z4YN62_MOUSE,Ataxin 7-like 3,Atxn7l3,Mus musculus (Mouse),EMSLSGLDNSKLEAIAQEIYADLVEDSCLGFCFEVHRAVKCGYFFL...,Not Available,"[IPR013246, IPR013243, IPR051078, ]"
85876,Z4YN74,Z4YN74_MOUSE,Transmembrane protein 11,Tmem11,Mus musculus (Mouse),MSVGMVSLSATDCYIVHEIYSGENAQDQFEYELEQALEAQYKYIVI...,Not Available,"[IPR026120, ]"
85878,Z4YN82,Z4YN82_MOUSE,Syntaphilin,Snph,Mus musculus (Mouse),MAMSLQGSRRASAGSRRRTSPPVSVRDAYGTSSLSSSSNSGSCKGS...,Not Available,"[IPR028197, ]"


In [8]:
df = df.drop_duplicates(subset=["Entry"]).reset_index(drop=True)
df.describe()

,Entry,Entry Name,Protein names,Gene Names,Organism,Sequence,EC number,InterPro,Domains
count,73862,73862,73862,73862,73862,73862,73862,73862,73862
unique,73862,73862,41087,33656,1,70969,1767,21481,21481
top,A0A075F5C6,A0A075F5C6_MOUSE,Olfactory receptor,Unknown,Mus musculus (Mouse),WFLEYSTSECHFYNGTQRVRLLERYFYNLEENLRFDSDVGEFRAVT...,Not Available,"[IPR000276, IPR017452, IPR000725, ]","IPR000276,IPR017452,IPR000725,"
freq,1,1,1741,2444,73862,12,63447,1160,1160


In [15]:
# df["Entry Name"] = df["Entry Name"].str.strip()
# df["Protein names"] = df["Protein names"].str.strip()
# df["Gene Names"] = df["Gene Names"].str.strip()

In [17]:
# Separating the dataframe into two - one for nodes and the other for edges(relationships)
nodes_df = df[['Entry', 'Entry Name', 'Protein names', 'Gene Names', 'InterPro']]
nodes_df.to_csv("nodes1.csv", index=False)
print("Completed")

Completed


In [ ]:
# Generating relationships using Jaccard Similarity
from itertools import combinations

edges = []

def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union > 0 else 0

for (idx1, row1), (idx2, row2) in combinations(df.iterrows(), 2):
    set1 = set(row1['InterPro'])
    set2 = set(row2['InterPro'])
    similarity = jaccard_similarity(set1, set2)
    if similarity > 0:
            edges.append((row1['Entry'], row2['Entry'], similarity))
            
edges_df = pd.DataFrame(edges, columns = ["Protein1", "Protein2", "Weight"])
edges_df.to_csv("edges.csv", index=False)
print("Completed")

In [ ]:
import csv
import numpy as np
from itertools import combinations

df['InterPro'] = df['InterPro'].apply(set)

# Define chunk size
chunk_size = 1000

# Get all combinations of rows in chunks
combinations_list = list(combinations(df.iterrows(), 2))
total_combinations = len(combinations_list)
print(f"Total combinations: {total_combinations}")

# Split combinations into manageable chunks
chunked_combinations = np.array_split(combinations_list, total_combinations // chunk_size + 1)

def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union > 0 else 0

with open("edges.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Protein1", "Protein2", "Weight"])

    for chunk in chunked_combinations:
        chunk_edges = []
        for (idx1, row1), (idx2, row2) in chunk:
            set1 = row1['InterPro']
            set2 = row2['InterPro']
            similarity = jaccard_similarity(set1, set2)
            if similarity > 0:
                chunk_edges.append((row1['Entry'], row2['Entry'], similarity))
        
        writer.writerows(chunk_edges)
        print(f"Processed and wrote chunk of size {len(chunk)}")


In [7]:
# Jaccard Similarity Function
def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union > 0 else 0


In [14]:
with open("edges2.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Protein1", "Protein2", "Weight"])  # Write header

    # Process combinations in manageable chunks
    batch_size = 500  # Adjust the batch size based on available memory
    combinations_generator = combinations(df.iterrows(), 2)  # Generate combinations

    for batch in range(0, len(df), batch_size):
        chunk_edges = []
        for (idx1, row1), (idx2, row2) in combinations(df[batch:batch + batch_size].iterrows(), 2):
            set1 = set(row1['InterPro'])
            set2 = set(row2['InterPro'])
            similarity = jaccard_similarity(set1, set2)
            if similarity > 0.8:
                chunk_edges.append((row1['Entry'], row2['Entry'], similarity))
        #break
        # Write the batch to the CSV file
        writer.writerows(chunk_edges)
        print(f"Processed batch {batch} - {batch + batch_size}")

Processed batch 0 - 500
Processed batch 500 - 1000
Processed batch 1000 - 1500
Processed batch 1500 - 2000
Processed batch 2000 - 2500
Processed batch 2500 - 3000
Processed batch 3000 - 3500
Processed batch 3500 - 4000
Processed batch 4000 - 4500
Processed batch 4500 - 5000
Processed batch 5000 - 5500
Processed batch 5500 - 6000
Processed batch 6000 - 6500
Processed batch 6500 - 7000
Processed batch 7000 - 7500
Processed batch 7500 - 8000
Processed batch 8000 - 8500
Processed batch 8500 - 9000
Processed batch 9000 - 9500
Processed batch 9500 - 10000
Processed batch 10000 - 10500
Processed batch 10500 - 11000
Processed batch 11000 - 11500
Processed batch 11500 - 12000
Processed batch 12000 - 12500
Processed batch 12500 - 13000
Processed batch 13000 - 13500
Processed batch 13500 - 14000
Processed batch 14000 - 14500
Processed batch 14500 - 15000
Processed batch 15000 - 15500
Processed batch 15500 - 16000
Processed batch 16000 - 16500
Processed batch 16500 - 17000
Processed batch 17000 - 

In [16]:
df2 = pd.read_csv("edges2.csv")
df2.describe()

,Weight
count,237724.000000
mean,0.974770
std,0.055031
min,0.809524
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [ ]:
ec_df = df[['Entry', 'EC number']]
ec_df.to_csv("data/ec_num.csv", index=False)
print("Completed")

Completed
